In [1]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0' :
  raise SystemError('GPU device not found')

print('Found GPU at : ', device_name)

Found GPU at :  /device:GPU:0


# 1. 모댈 만들기

In [13]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

> Conv2D는 다음과 같은 파라미터를 갖는다.
  - filters : 32의 출력 결과를 얻는다.
  - kernel_size = (3, 3)의 kernel을 사용하여 filtering하게 된다.
  - activation = 'relu'라는 활성화 함수를 이용한다.
  - input_shape = (28, 28, 1)의 입력 데이터를 이용한다.

> MaxPooling2D는 지정된 크기에서 가장 큰 값을 추출하는 코드로 현재는 2 x 2 공간 중 가장 큰 값을 추출하여 새로운 출력 데이터를 만들게 된다.

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 32)          0         
Total params: 9,568
Trainable params: 9,568
Non-trainable params: 0
_________________________________________________________________


> Conv2D의 output은 (input_shape - kernel_size + 1)의 결과를 얻는다.

> MaxPooling2D는 input_shape / filter_size로 이전 Conv2D의 output 결과를 input_shape으로 받아 들여 2, 2로 나눈 결과가 됨으로 26/2 = 13 이라는 결과를 얻게 된다.

In [15]:
model.add(layers.Flatten())

> 기존에 처리된 마지막 layer를 1차원으로 변환한다. 위의 경우 13, 13, 32의 output을 갖게 됨으로 1차원으로 변경하면 13 x 13 x 32 = 5408로 변경하게 되는 것이다.

In [16]:
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                51264     
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

# 2. 데이터 셋 얻기

## 2.1 데이터 셋 얻기

In [18]:
from keras.datasets import mnist

(train_data, train_layers), (test_data, test_layers) = mnist.load_data()

## 2.2 데이터 변환

In [19]:
X_train = train_data.reshape(60000, 28, 28, 1)
X_train = X_train.astype('float32')/255

X_test = test_data.reshape(10000, 28, 28, 1)
X_test = X_test.astype('float32')/255

In [20]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(train_layers)
y_test = to_categorical(test_layers)

# 3. 모델 학습 및 평가

## 3.1 모델 학습

In [21]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
with tf.device('/device:GPU:0') :
  model.fit(X_train, y_train, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 5s 9ms/step - loss: 0.2491 - accuracy: 0.9258
Epoch 2/5
469/469 [==============================] - 4s 8ms/step - loss: 0.0683 - accuracy: 0.9789
Epoch 3/5
469/469 [==============================] - 4s 8ms/step - loss: 0.0474 - accuracy: 0.9854
Epoch 4/5
469/469 [==============================] - 4s 8ms/step - loss: 0.0362 - accuracy: 0.9886
Epoch 5/5
469/469 [==============================] - 4s 8ms/step - loss: 0.0288 - accuracy: 0.9908


## 3.2 평가

In [23]:
loss, acc = model.evaluate(X_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0650 - accuracy: 0.9783


In [12]:
 5. 